In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\2\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\2\test'
checkpoint_path = r'C:\Users\24306\Desktop\NASNetMobile\2\NASNetMobile.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[80.0, 35.0, 13.0, 47.0, 55.0, 10.0, 93.0, 66.0, 7.0, 86.0, 7.0, 10.0, 47.0, 84.0, 96.0, 13.0, 100.0, 16.0, 40.0, 52.0, 84.0, 96.0, 93.0, 45.0, 32.0, 40.0, 80.0, 10.0, 93.0, 47.0, 13.0, 23.0, 37.0, 86.0, 96.0, 45.0, 86.0, 100.0, 84.0, 95.0, 16.0, 84.0, 96.0, 47.0, 100.0, 52.0, 32.0, 10.0, 66.0, 100.0, 84.0, 20.0, 52.0, 13.0, 66.0, 52.0, 4.0, 52.0, 37.0, 16.0, 13.0, 16.0, 7.0, 55.0, 4.0, 47.0, 45.0, 100.0, 95.0, 37.0, 96.0, 100.0, 95.0, 52.0, 93.0, 96.0, 84.0, 93.0, 10.0, 100.0, 47.0, 20.0, 80.0, 10.0, 37.0, 7.0, 80.0, 93.0, 20.0, 16.0, 47.0, 47.0, 96.0, 35.0, 55.0, 13.0, 80.0, 10.0, 10.0, 84.0, 37.0, 37.0, 37.0, 95.0, 4.0, 84.0, 37.0, 52.0, 96.0, 55.0, 84.0, 23.0, 52.0, 96.0, 30.0, 84.0, 45.0, 35.0, 13.0, 35.0, 86.0, 37.0, 45.0, 7.0, 80.0, 20.0, 16.0, 40.0, 10.0, 30.0, 55.0, 100.0, 55.0, 95.0, 96.0, 10.0, 37.0, 86.0, 37.0, 23.0, 37.0, 100.0, 7.0, 10.0, 7.0, 30.0, 20.0, 96.0, 20.0, 4.0, 39.0, 45.0, 86.0, 80.0, 52.0, 13.0, 13.0, 84.0, 40.0, 13.0, 84.0, 10.0, 66.0, 7.0, 95.0, 30.0, 84.0, 

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
NASNetMobile = tf.keras.applications.NASNetMobile(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
NASNetMobile.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = NASNetMobile(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (G  (None, 1056)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1057      
                                                                 
Total params: 4,270,773
Trainable params: 4,234,035
Non-trainable params: 36,738
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('NASNetMobile_loss_2.txt',history.history['loss'])
#要改
np.savetxt('NASNetMobile_valloss_2.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Sat Apr  8 08:28:13 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 76.8405

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\2\NASNetMobile.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\2\NASNetMobile.ckpt\assets


591/591 [==============================] - 187s 270ms/step - loss: 76.8405 - val_loss: 1565.7185
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 2.9260

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\2\NASNetMobile.ckpt\assets


...

591/591 [==============================] - 216s 365ms/step - loss: 0.0421 - val_loss: 0.0699
Epoch 66/300
591/591 [==============================] - 127s 214ms/step - loss: 0.0417 - val_loss: 0.1026
Epoch 67/300
591/591 [==============================] - 125s 211ms/step - loss: 0.0573 - val_loss: 0.1138
Epoch 68/300
591/591 [==============================] - 125s 211ms/step - loss: 0.0369 - val_loss: 0.1214
Epoch 69/300
591/591 [==============================] - 125s 212ms/step - loss: 0.0412 - val_loss: 0.1643
Epoch 70/300
591/591 [==============================] - 126s 212ms/step - loss: 0.0379 - val_loss: 0.0743
Epoch 71/300
591/591 [==============================] - 126s 213ms/step - loss: 0.0443 - val_loss: 0.1760
Epoch 72/300
591/591 [==============================] - 126s 214ms/step - loss: 0.0421 - val_loss: 0.1350
Epoch 73/300
591/591 [==============================] - 126s 212ms/step - loss: 0.0377 - val_loss: 0.1894
Epoch 74/300
591/591 [==============================] - 126

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\2\NASNetMobile.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\2\NASNetMobile.ckpt\assets


591/591 [==============================] - 213s 360ms/step - loss: 0.0370 - val_loss: 0.0618
Epoch 76/300
591/591 [==============================] - 131s 221ms/step - loss: 0.0437 - val_loss: 0.0955
Epoch 77/300
591/591 [==============================] - 129s 218ms/step - loss: 0.0421 - val_loss: 0.1259
Epoch 78/300
591/591 [==============================] - 128s 216ms/step - loss: 0.0438 - val_loss: 0.1002
Epoch 79/300
591/591 [==============================] - 128s 216ms/step - loss: 0.0431 - val_loss: 0.1022
Epoch 80/300
591/591 [==============================] - 129s 219ms/step - loss: 0.0364 - val_loss: 0.1137
Epoch 81/300
591/591 [==============================] - 128s 217ms/step - loss: 0.0345 - val_loss: 0.1694
Epoch 82/300
591/591 [==============================] - 129s 218ms/step - loss: 0.0355 - val_loss: 0.1566
Epoch 83/300
591/591 [==============================] - 129s 217ms/step - loss: 0.0593 - val_loss: 0.2481
Epoch 84/300
591/591 [==============================] - 129

Epoch 152/300
591/591 [==============================] - 131s 222ms/step - loss: 0.0314 - val_loss: 0.1583
Epoch 153/300
591/591 [==============================] - 132s 223ms/step - loss: 0.0291 - val_loss: 0.1428
Epoch 154/300
591/591 [==============================] - 132s 223ms/step - loss: 0.0298 - val_loss: 0.1709
Epoch 155/300
591/591 [==============================] - 132s 223ms/step - loss: 0.0263 - val_loss: 0.1421
Epoch 156/300
591/591 [==============================] - 132s 224ms/step - loss: 0.0339 - val_loss: 0.1264
Epoch 157/300
591/591 [==============================] - 132s 224ms/step - loss: 0.0310 - val_loss: 0.1133
Epoch 158/300
591/591 [==============================] - 132s 224ms/step - loss: 0.0262 - val_loss: 0.1123
Epoch 159/300
591/591 [==============================] - 133s 224ms/step - loss: 0.0342 - val_loss: 0.1189
Epoch 160/300
591/591 [==============================] - 132s 224ms/step - loss: 0.0308 - val_loss: 0.1208
Epoch 161/300
591/591 [==============